In [1]:
import re
import pymorphy2
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [2]:
tags = ('NOUN', 'ADJ', 'COMP', 'VERB', 'INFN', 'PRT', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ')

In [3]:
def lemmatization(word):
    w = morph.parse(word)
    l = w[0].normal_form
    return (l)

In [4]:
def only_one_word(phrase):
    to_search = {'token': None, 'lemma': None, 'pos': None}
    if phrase[0].upper() in tags:
        phrase = phrase[0].upper()
        to_search['pos'] = phrase
        #print('ищем в столбце pos')
    else:
        phrase = phrase[0].lower()
        if re.search(r'[a-z]', phrase):
            print('Латинские символы могут быть использованы только для частеречных тегов')
            return None
        elif phrase[0] == '"':
            to_search['token'] = phrase[1:-1]
            #print('ищем в столбце token')
        else:
            l = lemmatization(phrase)
            to_search['lemma'] = l
    return to_search

In [5]:
def token_and_tag(phrase):
    if re.search(r'[a-z]', phrase[0].lower()):
        print('Формат введенного запроса неверный')
        return None
    else:
        to_search = {'token': None, 'lemma': None, 'pos': None}
        w = phrase[0].lower() #слово
        if w[0] == '"':
            to_search['token'] = w[1:-1]
        else:
            l = lemmatization(w)
            to_search['lemma'] = l
        p = phrase[1].upper()
        if p not in tags:
            print('Указан неправильный частеречный тег')
            return None
        else:
            to_search['pos'] = p
            return to_search

In [99]:
def main_search(q):
    if re.search(r'[^a-zA-Zа-яА-Я +"]', q):
        print('В запросе присутствуют некорректные символы')
        return None
    else:
        request = []
        q = q.split()
        if len(q) > 3:
            print('Превышена длина запроса')
        elif len(q) == 1:    # значит в запросе одно слово
            q1 = q[0].split('+')
            if len(q1) == len(q):    # либо только токен, либо только тег
                result = only_one_word(q1)
                if result is not None:
                    request.append(result)
            else:    # в запросе токен и тег
                result = token_and_tag(q1)
                if result is not None:
                    request.append(result)
        else:    # в запросе не одно слово
            for element in q:
                q1 = element.split('+')
                if len(q1) == 1:    # либо только токен, либо только тег
                    result = only_one_word(q1)
                    if result is not None:
                        request.append(result)
                    else:
                        request = []
                        break
                else:    # в запросе токен и тег
                    result = token_and_tag(q1)
                    if result is not None:
                        request.append(result)
                    else:
                        request = []
                        break
    return request

In [118]:
q = 'мыла+VERB'    # сюда введите запрос
q_list = main_search(q)

In [119]:
print(q_list)

[{'token': None, 'lemma': 'мыло', 'pos': 'VERB'}]


In [105]:
import sqlite3

In [106]:
conn = sqlite3.connect('poems_corpus.db')
cur = conn.cursor()

In [107]:
def token_and_pos(dct):
  query = """
  SELECT token_id, sent_id
  FROM tokens
  WHERE token = ? AND pos = ?
  """
  cur.execute(query, (dct['token'], dct['pos']))
  result = cur.fetchall()
  return result

In [108]:
def lemma_and_pos(dct):
  query = """
  SELECT token_id, sent_id
  FROM tokens
  WHERE lemma = ? AND pos = ?
  """
  cur.execute(query, (dct['lemma'], dct['pos']))
  result = cur.fetchall()
  return result

In [109]:
def only_token(dct):
  query = """
  SELECT token_id, sent_id
  FROM tokens
  WHERE token = ?
  """
  cur.execute(query, (dct['token'],))
  result = cur.fetchall()
  return result

In [110]:
def only_lemma(dct):
  query = """
  SELECT token_id, sent_id
  FROM tokens
  WHERE lemma = ?
  """
  cur.execute(query, (dct['lemma'],))
  result = cur.fetchall()
  return result

In [111]:
def only_pos(dct):
  query = """
  SELECT token_id, sent_id
  FROM tokens
  WHERE pos = ?
  """
  cur.execute(query, (dct['pos'],))
  result = cur.fetchall()
  return result

In [112]:
def get_for_next(result, next):
  sentences = []
  if next['token']:
    item = 'token'
    if next['pos']:
      item += ' = ? AND pos'
  elif next['lemma']:
    item = 'lemma'
    if next['pos']:
      item += ' = ? AND pos'
  elif next['pos']:
    item = 'pos'
  t_query = f"""
    SELECT token_id, sent_id
    FROM tokens
    WHERE {item} = ? AND token_id = ? AND sent_id = ?
    """
  for token_id, sent_id in result:
    next_id = token_id + 1
    if ' ' in item:
      items = item.split(' = ? AND ')
      cur.execute(t_query, (next[items[0]], next[items[1]], next_id, sent_id))
    else:
      cur.execute(t_query, (next[item], next_id, sent_id))
    cur_sent = cur.fetchone()
    if cur_sent:
      sentences.append(cur_sent)
  return sentences

In [113]:
def sort_sentences(q_list):
  if q_list[0]['token'] and q_list[0]['pos']: # если в словарь токена записывать не пустые строки, а None
    result = token_and_pos(q_list[0]) # id токенов и предложений
  elif q_list[0]['lemma'] and q_list[0]['pos']:
    result = lemma_and_pos(q_list[0])
  elif q_list[0]['token'] and not q_list[0]['pos']:
    result = only_token(q_list[0])
  elif q_list[0]['lemma'] and not q_list[0]['pos']:
    result = only_lemma(q_list[0])
  else:
    result = only_pos(q_list[0])
  #print(result)
  if result and len(q_list) > 1:
    sentences = get_for_next(result, q_list[1])
    #print(sort_sentences)
    if len(q_list) == 3:
      sentences = get_for_next(sentences, q_list[2])
      sentences = [obj[1] for obj in sentences]
      #print(sentences)
      return sentences
    else:
      sentences = [obj[1] for obj in sentences]
      return sentences # получаем список айдишников предложений
  if result:
    result = [obj[1] for obj in result]
  return result

In [114]:
print(sort_sentences(q_list))

[543, 3618, 5697, 5698, 8092, 8130]


In [115]:
ids = sort_sentences(q_list)

In [116]:
def get_sentences(ids):
    if len(ids) == 1:
        ids = f'({ids[0]})'
    else:
        ids = tuple(ids)
    posts_query = f'''
        SELECT poet, title, sent FROM sentences
        JOIN poems_to_info ON sentences.id_sent== poems_to_info.id_sent
        JOIN info ON poems_to_info.id_info == info.id_info
        WHERE sentences.id_sent in {ids}
    '''
    cur = conn.cursor()
    cur.execute(posts_query)
    result = cur.fetchall()
    return result

In [117]:
get_sentences(ids)

[('Владимир Маяковский',
  'Бюрократиада',
  'Все предлагали умно и трезво Вдвое урезывать Втрое урезывать Строчил секретарь от работы в мыле постановили слушали слушали Всю ночь над машинкой склонившись низко резолюции переписывала и переписывала машинистка'),
 ('Сергей Есенин',
  'Письмо к женщине',
  'Не знали вы что в сонмище людском Я был как лошадь загнанная в мыле Пришпоренная смелым ездоком'),
 ('Владимир Маяковский',
  'Сказка о Пете, толстом ребенке, и о Симе, который тонкий',
  'Сима чистый чище мыла'),
 ('Владимир Маяковский',
  'Сказка о Пете, толстом ребенке, и о Симе, который тонкий',
  'Мылся сам и мама мыла'),
 ('Владимир Маяковский',
  'Четырёхэтажная халтура',
  'А вокруг скачут критики в мыле и пене Здорово пишут писатели братцы'),
 ('Владимир Маяковский',
  'Что такое хорошо и что такое плохо',
  'Если мальчик любит мыло и зубной порошок этот мальчик очень милый поступает хорошо')]